# Inicialização do ambiente

In [1]:
# Essentials
import pandas as pd
import numpy as np

import datetime as dt
import gc
import math
import warnings, time
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
%matplotlib inline

# Plots
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

# Stats
from scipy import stats
from scipy.stats import skew, norm, rankdata, pearsonr
from sklearn.impute import KNNImputer
import ppscore as pps

# Estebelece limites para visualização no notebook
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',500)

# Limita a 3 casas decimais a apresentação das variaveis tipo float
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))

# Handle date time conversions between pandas and matplotlib
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# Coleta de dados do banco SQL

In [ ]:
server = datausuario.getHost('userX')
usuario = datausuario.getUser('userX')
chave = datausuario.getPass('userX')
database = 'ds_database'

### Estabelece conexão com o banco

In [ ]:
import pyodbc
from pyodbc import Error

try:
    conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server}; \
                         Server='+server+'; \
                         Port=1433; \
                         Database='+database+'; \
                         Trusted_Connection=yes; \
                         uid='+usuario+'; \
                         pwd='+chave)
    
    db_name = conn.getinfo(pyodbc.SQL_DBMS_NAME)
    print("You're connected to database: ", db_name)
    db_ver = conn.getinfo(pyodbc.SQL_DBMS_VER)
    print("Databse version: ", db_ver)
    
    cursor = conn.cursor()

except Error as e:
    print("Error while connecting to MSSQL", e)
    
del server, usuario, chave, database
del db_name, db_ver
gc.collect();

### Faz a coleta dos dados e cria dataframe

In [ ]:
query = """SELECT * FROM ds_database.dbo.historico
           WHERE itemref = 'Temperature'
                 AND data <= '2020-12-31 23:59:59'"""

df_hist = pd.read_sql(query, con=conn)

In [ ]:
df_hist.sort_values(by=['id'], ignore_index=True, inplace=True)

### Encerra conexão com o banco

In [ ]:
# Encerra conexão com o banco MSSQL
cursor.close()
conn.close()
del cursor, conn, query
gc.collect();

# Cria dataset de referencia para estudos

In [ ]:
df_hist['itemref'].unique()

In [ ]:
dic = {'Temperature': 'temp_int',
       'Humidity': 'umid_int',
       'Output Current - Line 1': 'out_curr1',
       'Output Current - Line 2': 'out_curr2',
       'Output Current - Line 3': 'out_curr3',
       'Output Percent Load - Line 1': 'out_load1',
       'Output Percent Load - Line 2': 'out_load2',
       'Output Percent Load - Line 3': 'out_load3',
       'Output Power - Line 1': 'out_powr1',
       'Output Power - Line 2': 'out_powr2',
       'Output Power - Line 3': 'out_powr3',
       'Output Power': 'out_powr'}

In [ ]:
df_hist['itemref'] = df_hist['itemref'].map(dic)
df_hist['host'] = df_hist['host'] + '_' + df_hist['itemref']
df_hist.drop(['id', 'utc', 'itemref'], axis=1, inplace=True)
df_hist

In [ ]:
# Modifica dataset para ajustá-lo ao modelo desejado para os estudo
df_hist = df_hist.groupby(['data', 'host'])['value'].aggregate('mean').unstack()
df_hist.head(3)

In [ ]:
# Fazendo resample dos registros considerando consolidações em 10 minutos
df1 = df_hist.resample('10min').mean()

# Análise gráfica do comportamento das variáveis no tempo

In [2]:
# Lista de instrumentos de medição e localização
df_host = pd.read_csv('../data/df_host.csv', sep=';', encoding='utf-8', index_col=['host'])  

In [5]:
# Pré seta definições de tamanho para apresentação de gráficos
f, ax = plt.subplots(nrows=4, ncols=5)
plt.rcParams['figure.figsize'] = [20, 20]
plt.subplots_adjust(wspace=0.25, hspace=0.23)
plt.close('all')

In [9]:
# Grafico
fileout= '../fig/{}.jpg'

for inst in df1columns:
    host = inst.replace('_temp_int', '')
    site = df_host.loc[host, 'site']
    host = host+' - '+site
    
    # mapeia ambiente para construção de 5 gráficos 
    f, axes = plt.subplots(nrows=4, ncols=5)
    plt.rcParams['figure.figsize'] = [20, 20]
    plt.subplots_adjust(wspace=0.25, hspace=0.23)

    ax1 = plt.subplot2grid((4, 5), (0, 0), colspan=5, rowspan=2)
    ax2 = plt.subplot2grid((4, 5), (2, 0), colspan=5, rowspan=1)
    ax3 = plt.subplot2grid((4, 5), (3, 0), colspan=2, rowspan=1)
    ax4 = plt.subplot2grid((4, 5), (3, 2), colspan=2, rowspan=1)
    ax5 = plt.subplot2grid((4, 5), (3, 4))

    # Gráfico 1 - Médias diária e semanal
    x = df1[inst].resample('1H').apply([np.mean])
    y = df1[inst].resample('W').apply([np.mean])

    ax1.plot(x, label='Média a cada hora')
    ax1.plot(y, label='Média semanal')
    ax1.set_title('Médias de temperatura de '+host, fontsize=14)
    ax1.set_xlabel('')
    ax1.set_ylabel('Temperatura Interna', fontsize=14)
    ax1.legend(fontsize=12)
    ax1.grid(color='lightgray', linestyle='-', linewidth=0.5)

    # Grafico 2 - Estatisicas de cada semana
    df_aux1 = df1[inst].resample('10min').apply([np.mean])
    (df_aux1.assign(periodo=lambda df: df_aux1.index.weekofyear)
       .groupby('periodo')['mean'].agg(["mean", "median", "min", "max"])
       .plot(ax=ax2, marker="o",linewidth=1))
    ax2.set_title('Estatísticas a cada semana', fontsize=14)
    ax2.set_xlabel('')
    ax2.set_ylabel('Temperatura Interna', fontsize=14)

    # Gráfico 3 - Histograma
    df1[inst].hist(bins = 30, ax=ax3)
    ax3.set_xlabel('Temperaturas', fontsize=12)
    ax3.set_ylabel('Ocorrências', fontsize=12)
    ax3.set_title('Distribuição no periodo todo', fontsize=14)
    
        # Gráfico 4 - Boxplot
    sns.boxplot(x=df1.index.month, y=inst, data=df, ax=ax4, orient="v", palette="Set2")
    ax4.set_xlabel('Meses', fontsize=14)
    ax4.set_ylabel('')
    ax4.set_title('Distribuição nos meses', fontsize=14)

    # Gráfico 5 - Distribuição
    sns.stripplot(x=df1.index.month, y=inst, data=df, ax=ax5)
    ax5.set_xlabel('Meses', fontsize=14)
    ax5.set_ylabel('')
    ax5.set_title('Distribuição nos meses', fontsize=14)
                           
    # Salva gráfico em arquivo jpg
    plt.savefig(fileout.format(inst), format='jpg')

    plt.show();
  
    